In [53]:
print(classification_report(y_test, preds))
#print(fps_adam)
print('mAP: ' + mAP_adam)
print ("Total number of parameters: " + str(params))

              precision    recall  f1-score   support

           0       1.00      0.93      0.97       300
           1       0.87      1.00      0.93       300
           2       1.00      0.92      0.96       300

   micro avg       0.95      0.95      0.95       900
   macro avg       0.96      0.95      0.95       900
weighted avg       0.96      0.95      0.95       900
 samples avg       0.95      0.95      0.95       900

mAP: 0.9674074074074074
Total number of parameters: 50147


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch import Tensor
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import matplotlib.image as mpimg
import pandas as pd
from numpy import prod
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelBinarizer
import time
import glob
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

import warnings
"""Optional[...] is a shorthand notation for Union[..., None], 
telling the type checker that either an object of the specific type is required, 
or None is required. ... stands for any valid type hint, 
including complex compound types or a Union[] of more types. 
Whenever you have a keyword argument with default value None, 
you should use Optional."""
from torch.jit.annotations import Optional, Tuple

from torch.hub import load_state_dict_from_url

from torchsummary import summary
from tqdm import tqdm

In [3]:
import torchvision.models as models
model = models.googlenet(pretrained = True)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', torch.cuda.get_device_name(torch.cuda.current_device()))

Device: GeForce RTX 2080 Ti


In [5]:
torch.cuda.is_available()

True

In [6]:
for param in model.parameters():
    param.requires_grad = False 

In [7]:
model = nn.Sequential(*list(model.children())[0:3])
model.lrn = nn.LocalResponseNorm(5, alpha = 9.9999997e-05)
model.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True, padding=0, dilation=1)
#[(W−K+2P)/S]+1
#model = nn.Sequential(*[model[i] for i in range(4)])
print(model)

Sequential(
  (0): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lrn): LocalResponseNorm(5, alpha=9.9999997e-05, beta=0.75, k=1.0)
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
)


In [8]:
class Inception(nn.Module):

    def __init__(self, in_channels, ch1x1, ch3x3red, pool_proj,
                 conv_block=None):
        super(Inception, self).__init__()
        if conv_block is None:
            conv_block = BasicConv2d
            
        self.branch1 = conv_block(in_channels, ch1x1, kernel_size=1)

        self.branch2 = conv_block(in_channels, ch3x3red, kernel_size=1)


    def _forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)


        outputs = [branch1, branch2]
        return outputs

    def forward(self, x):
        outputs = self._forward(x)
        return torch.cat(outputs, 1)

class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)

    def forward(self, x):
        x = self.conv(x)
        x = F.relu(x, inplace=False)
        return x

In [9]:
blocks = [BasicConv2d, Inception]
conv_block = blocks[0]
inception_block = blocks[1]

In [10]:
model.inception3a = inception_block(64, 8, 16, 28)

In [11]:
model.avgpool = nn.AdaptiveAvgPool2d(output_size=(22, 22))
model.dropout = nn.Dropout(p = 0.20000001)
model.fc = nn.Sequential(OrderedDict([
    ('flatten', nn.Flatten()),
    ('fc1', nn.Linear(11616, 3)),
    ('output', nn.LogSoftmax(dim = 1))
    ]))
model = model.to(device)
summary(model, input_size=(3, 224, 224), batch_size=128, device = str(torch.device("cuda")))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 112, 112]           9,408
       BatchNorm2d-2        [128, 64, 112, 112]             128
       BasicConv2d-3        [128, 64, 112, 112]               0
         MaxPool2d-4          [128, 64, 56, 56]               0
            Conv2d-5          [128, 64, 56, 56]           4,096
       BatchNorm2d-6          [128, 64, 56, 56]             128
       BasicConv2d-7          [128, 64, 56, 56]               0
 LocalResponseNorm-8          [128, 64, 56, 56]               0
         MaxPool2d-9          [128, 64, 28, 28]               0
           Conv2d-10           [128, 8, 28, 28]             512
      BasicConv2d-11           [128, 8, 28, 28]               0
           Conv2d-12          [128, 16, 28, 28]           1,024
      BasicConv2d-13          [128, 16, 28, 28]               0
        Inception-14          [128, 24,

In [12]:
model

Sequential(
  (0): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lrn): LocalResponseNorm(5, alpha=9.9999997e-05, beta=0.75, k=1.0)
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (branch2): BasicConv2d(
      (conv): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(22, 22))
  (dropout): Dropout(p=0.20000001, inplace=False)
  (fc

In [13]:
model.classifier = nn.Sequential(*list(model.children())[3:9])

In [14]:
model.classifier

Sequential(
  (0): LocalResponseNorm(5, alpha=9.9999997e-05, beta=0.75, k=1.0)
  (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (2): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (branch2): BasicConv2d(
      (conv): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
  )
  (3): AdaptiveAvgPool2d(output_size=(22, 22))
  (4): Dropout(p=0.20000001, inplace=False)
  (5): Sequential(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=11616, out_features=3, bias=True)
    (output): LogSoftmax(dim=1)
  )
)

In [15]:
def get_num_correct(preds, labels):
      return preds.argmax(dim=1).eq(labels).sum().item()

In [16]:
# Read in the hyper-parameters and return a Run namedtuple containing all the 
# combinations of hyper-parameters
class RunBuilder():
    @staticmethod
    def get_runs(params):
        
        Run = namedtuple("Run", params.keys())
        
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
            
        return runs

In [17]:
# Helper class, help track loss, accuracy, epoch time, run time, 
# hyper-parameters etc. Also record to TensorBoard and write into csv, json!@!
class RunManager():
    def __init__(self):
        
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        # tracking every run count, run data, hyper-params used, time
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
    
    def begin_run(self, run, network, loader):
        
        self.run_start_time = time.time()
        
        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment = f'-{run}')
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
        #self.tb.add_image('images', grid)
        #self.tb.add_graph(self.network, images)
        
    def end_run(self, model):
        
        MODEL_PATH = "./weights/model_{}.pt".format(self)
        torch.save(model, MODEL_PATH)
        
        self.tb.close()
        self.epoch_count = 0
       
       # zero epoch count, loss, accuracy, 
    def begin_epoch(self):
        self.epoch_start_time = time.time()

        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        
    # 
    def end_epoch(self, model):
        # calculate epoch duration and run duration(accumulate)
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        # record epoch loss and accuracy
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)

        # Record epoch loss and accuracy to TensorBoard 
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

        # Record params to TensorBoard
       # for name, param in self.network.named_parameters():
       #     self.tb.add_histogram(name, param, self.epoch_count)
       #     self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
    

        # Write into 'results' (OrderedDict) for all run related data
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        
        MODEL_PATH = "./weights/model_{}.pt".format(self.epoch_count)
        torch.save(model, MODEL_PATH)
        
        # Record hyper-params into 'results'
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient = 'columns')

        # display epoch information and show progress
        clear_output(wait=True)
        display(df)

      # accumulate loss of batch into entire epoch loss
    def track_loss(self, loss):
        # multiply batch size so variety of batch sizes can be compared
        self.epoch_loss += loss.item() * self.loader.batch_size

      # accumulate number of corrects of batch into entire epoch num_correct
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)
    
    @torch.no_grad()
    def _get_num_correct(self, preds, lables):
        return preds.argmax(dim = 1).eq(labels).sum().item()
    
    def save(self, fileName):
        
        pd.DataFrame.from_dict(
        self.run_data,
        orient = "columns").to_csv(f'{fileName}.csv')
        
        
        
        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [18]:
train_set = torchvision.datasets.ImageFolder(
    root = './Data_cleaned/train'
    ,transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

In [19]:
%%time
from tqdm import tqdm_notebook

loader = torch.utils.data.DataLoader(
    train_set, batch_size = 512, num_workers = 2, pin_memory = True
)
data = next(iter(loader))
mean = data[0].mean()
std = data[0].std()
mean, std

CPU times: user 246 ms, sys: 149 ms, total: 395 ms
Wall time: 7.53 s


(tensor(0.4738), tensor(0.2598))

In [20]:
train_set_normal = torchvision.datasets.ImageFolder(
    root = './Data_cleaned/train'
    ,transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)

In [21]:
trainsets = {
    'not_normal': train_set,
    'normal' : train_set_normal
}

In [22]:
network = nn.Sequential(*list(model.children())[0:3])
network.classifier = model.classifier

In [23]:
networks = {
    'batch_norm': network
}

In [24]:
CURRENT_DIR = '/home/vadim/testovoe/dev/'
test_df = pd.read_csv(CURRENT_DIR+"Data/text.txt", sep="\s+", header=None, names=["name", "category"])

In [25]:
def get_pad_width(im, new_shape, is_rgb=True):
    pad_diff = new_shape - im.shape[0], new_shape - im.shape[1]
    t, b = math.floor(pad_diff[0]/2), math.ceil(pad_diff[0]/2)
    l, r = math.floor(pad_diff[1]/2), math.ceil(pad_diff[1]/2)
    if is_rgb:
        pad_width = ((t,b), (l,r), (0, 0))
    else:
        pad_width = ((t,b), (l,r))
    return pad_width

def preprocess_image(image_path, desired_size=224):
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.LANCZOS)
    
    return im

In [26]:
N = test_df.shape[0]
x_test = np.empty((N, 224, 224, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(test_df['name'])):
    x_test[i, :, :, :] = preprocess_image(
         f'/home/vadim/testovoe/dev/{image_id}'
    )

100%|██████████| 900/900 [00:02<00:00, 414.07it/s]


In [27]:
y_test = pd.get_dummies(test_df['category']).values

In [28]:
network

Sequential(
  (0): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (classifier): Sequential(
    (0): LocalResponseNorm(5, alpha=9.9999997e-05, beta=0.75, k=1.0)
    (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (2): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (branch2): BasicConv2d(
        (conv): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (3): AdaptiveAvgPool2d(output_size=(22, 22))
    (4): Dropout(p=0.20000001,

In [ ]:
%%time
# Test different configurations
# for every run [value] that is going to be used e.g [.001, .01] = two runs
# the bigger the batch the less gradient updates steps made
# implement callback
params = OrderedDict(
    lr = [.001],
    # max batch size 352
    batch_size = [128],
    weight_decay = [6.0000001e-06],
    num_workers = [1],
    device = ["cuda"],
    trainset = ["normal"],
    # try all the values in the dict network1, network2
    network = list(networks.keys())
)
m = RunManager()
# active run or current run
# this thing detecting RuntimeError: Function 'LogSoftmaxBackward' returned nan values in its 0th output. 
# if k = 0 in LRN layers
torch.autograd.set_detect_anomaly(True)
for run in RunBuilder.get_runs(params):
    
    device = torch.device(run.device)
    # redefine the network
    network = networks[run.network].to(device)
    network.train()
    loader = DataLoader(trainsets[run.trainset], batch_size = run.batch_size, num_workers = run.num_workers, 
                       pin_memory = True) 
    optimizer = optim.Adam(network.classifier.parameters(), lr = run.lr, weight_decay = run.weight_decay, amsgrad=True) 
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)
    
    m.begin_run(run, network, loader)
    for epoch in range(300):
        m.begin_epoch()
        for batch in loader:
            #network.train()
            images = batch[0].to(device)
            labels = batch[1].to(device)
            for p in network.parameters(): p.grad = None
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            #7.9
            #optimizer.zero_grad()
            #8 sec
            
            loss.backward() # Calculate gradients
            optimizer.step() # Update Weights
            m.track_loss(loss)
            scheduler.step() 
            m.track_num_correct(preds, labels)
        m.end_epoch(network)
    m.end_run(network)
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,weight_decay,num_workers,device,trainset,network
0,1,1,0.950129,0.935096,55.798757,56.386567,0.001,128,0.000006,1,cuda,normal,batch_norm
1,1,2,3.080913,0.272820,56.180495,112.584605,0.001,128,0.000006,1,cuda,normal,batch_norm
2,1,3,2.522101,0.332233,54.493393,167.098216,0.001,128,0.000006,1,cuda,normal,batch_norm
3,1,4,1.036301,0.332572,55.348439,222.463122,0.001,128,0.000006,1,cuda,normal,batch_norm
4,1,5,1.763039,0.333066,55.392844,277.873277,0.001,128,0.000006,1,cuda,normal,batch_norm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1,295,0.048324,0.983712,53.825144,15974.429213,0.001,128,0.000006,1,cuda,normal,batch_norm
295,1,296,0.048999,0.983157,53.989328,16028.443078,0.001,128,0.000006,1,cuda,normal,batch_norm
296,1,297,0.048734,0.983311,54.265919,16082.733626,0.001,128,0.000006,1,cuda,normal,batch_norm
297,1,298,0.047343,0.983712,53.828166,16136.586855,0.001,128,0.000006,1,cuda,normal,batch_norm


In [30]:
pd.DataFrame.from_dict(m.run_data).sort_values("accuracy", ascending = False)

NameError: name 'm' is not defined

In [214]:
MODEL_PATH = "./weights/model_297.pt"
model = torch.load(MODEL_PATH)

In [215]:
networks = {
    'batch_norm': network,
    'trained_net': model
}

In [216]:
%%time
# Test different configurations
# for every run [value] that is going to be used e.g [.001, .01] = two runs
# the bigger the batch the less gradient updates steps made
# implement callback
params = OrderedDict(
    lr = [.0003],
    # max batch size 352
    batch_size = [128],
    weight_decay = [3.0000001e-06],
    num_workers = [1],
    device = ["cuda"],
    trainset = ["normal"],
    # try all the values in the dict network1, network2
    #network = list(networks.keys())
    network = ["trained_net"]
)
m = RunManager()
# active run or current run
# this thing detecting RuntimeError: Function 'LogSoftmaxBackward' returned nan values in its 0th output. 
# if k = 0 in LRN layers
torch.autograd.set_detect_anomaly(True)
for run in RunBuilder.get_runs(params):
    
    device = torch.device(run.device)
    # redefine the network
    network = networks[run.network].to(device)
    network.train()
    loader = DataLoader(trainsets[run.trainset], batch_size = run.batch_size, num_workers = run.num_workers, 
                       pin_memory = True) 
    optimizer = optim.Adam(network.parameters(), lr = run.lr, weight_decay = run.weight_decay, amsgrad=True) 
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)
    
    m.begin_run(run, network, loader)
    for epoch in range(300):
        m.begin_epoch()
        for batch in loader:
            #network.train()
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            #7.9
            #optimizer.zero_grad()
            #8 sec
            for p in network.parameters(): p.grad = None
            loss.backward() # Calculate gradients
            optimizer.step() # Update Weights
            m.track_loss(loss)
            scheduler.step() 
            m.track_num_correct(preds, labels)
        m.end_epoch(network)
    m.end_run(network)
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,weight_decay,num_workers,device,trainset,network
0,1,1,0.077972,0.988679,54.115081,54.727784,0.0003,128,0.000003,1,cuda,normal,trained_net
1,1,2,0.933191,0.941019,54.660468,109.404345,0.0003,128,0.000003,1,cuda,normal,trained_net
2,1,3,0.231535,0.941142,53.510615,162.931849,0.0003,128,0.000003,1,cuda,normal,trained_net
3,1,4,0.059162,0.979147,53.492336,216.445287,0.0003,128,0.000003,1,cuda,normal,trained_net
4,1,5,0.059811,0.979856,53.873811,270.333647,0.0003,128,0.000003,1,cuda,normal,trained_net
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,296,0.005956,0.998735,53.697124,15858.582165,0.0003,128,0.000003,1,cuda,normal,trained_net
296,1,297,0.005722,0.998859,53.559353,15912.168745,0.0003,128,0.000003,1,cuda,normal,trained_net
297,1,298,0.005811,0.998797,53.699200,15965.893756,0.0003,128,0.000003,1,cuda,normal,trained_net
298,1,299,0.005987,0.998797,53.634737,16019.551512,0.0003,128,0.000003,1,cuda,normal,trained_net


CPU times: user 57min 13s, sys: 15min 31s, total: 1h 12min 45s
Wall time: 4h 27min 53s


In [217]:
pd.DataFrame.from_dict(m.run_data).sort_values("accuracy", ascending = False)

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,weight_decay,num_workers,device,trainset,network
299,1,300,0.005454,0.999167,53.566628,16073.144558,0.0003,128,0.000003,1,cuda,normal,trained_net
291,1,292,0.005719,0.998982,53.586104,15643.860402,0.0003,128,0.000003,1,cuda,normal,trained_net
285,1,286,0.005670,0.998982,53.653476,15322.211100,0.0003,128,0.000003,1,cuda,normal,trained_net
275,1,276,0.005793,0.998982,53.477520,14785.783270,0.0003,128,0.000003,1,cuda,normal,trained_net
290,1,291,0.005906,0.998982,53.647564,15590.250579,0.0003,128,0.000003,1,cuda,normal,trained_net
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,1,7,0.046618,0.985378,53.531803,377.502581,0.0003,128,0.000003,1,cuda,normal,trained_net
4,1,5,0.059811,0.979856,53.873811,270.333647,0.0003,128,0.000003,1,cuda,normal,trained_net
3,1,4,0.059162,0.979147,53.492336,216.445287,0.0003,128,0.000003,1,cuda,normal,trained_net
2,1,3,0.231535,0.941142,53.510615,162.931849,0.0003,128,0.000003,1,cuda,normal,trained_net


In [41]:
MODEL_PATH = "./weights/model_292.pt"
model = torch.load(MODEL_PATH)

In [42]:
test_set = torchvision.datasets.ImageFolder(
    root = './Data/test'
    ,transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)

In [43]:
test_loader = DataLoader(test_set, batch_size = 128, num_workers = 1, 
                       pin_memory = True) 

In [44]:
def test(model, optimizer, test_loader):
    """
    Args:
        model : network
        test_loader : pytorch test loader
    Returns:
        list : predicted values
    """
    
    model = torch.load(MODEL_PATH)
    optimizer = optimizer
    preds = []

    with torch.no_grad():
        for batch in test_loader:
            images = batch[0].to(device)
            labels = batch[1].to(device)
            output = model(images)
            pred = output.argmax(dim=1, keepdim=True)
            preds.extend(pred.cpu().numpy())

    return preds

In [45]:
optimizer = optim.Adam(network.parameters(), lr = 0.0003, weight_decay = 0.000003)

# get predictions
predictions = test(torch.load(MODEL_PATH), optimizer, test_loader)

In [46]:
preds = predictions

In [47]:
def extractDigits(lst): 
    return list(map(lambda el:el.tolist(), lst)) 
extractDigits(preds)
flat_list = []
flat_list = [item for sublist in preds for item in sublist]

In [49]:
lb = LabelBinarizer()
preds = lb.fit_transform(flat_list)
preds

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [50]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       1.00      0.93      0.97       300
           1       0.87      1.00      0.93       300
           2       1.00      0.92      0.96       300

   micro avg       0.95      0.95      0.95       900
   macro avg       0.96      0.95      0.95       900
weighted avg       0.96      0.95      0.95       900
 samples avg       0.95      0.95      0.95       900



In [51]:
mAP_adam = str(average_precision_score(y_test, preds, average="samples"))
print(average_precision_score(y_test, preds, average="samples"))

0.9674074074074074


In [52]:
params = sum([param.nelement() for param in model.parameters()])

In [ ]:
!shutdown